## Purpose: Extract particle size distribution data from .txt generated by GGU Sieve to make it fit for import to GeODin

In [183]:
# load needed packages

import pandas as pd
import os
import pprint

In [253]:
# load list of samples

samples_path = r'Y:\2022\Projekte_364\364_22_012_WPD_Germany_Gennaker SI\4_Data\8_Sample_Geodin_Import\Sample_List_All.xlsx'

samples_df = pd.read_excel(samples_path, usecols=['Location', 'Sample', 'ID', 'Sample from [m]'])

print(samples_df.head())

print(samples_df.shape)

    Location Sample                              ID  Sample from [m]
0  GN_A05_BH    DS1  GN_A05_BH : DS1 (0,00 - 0,50m)             0.00
1  GN_A05_BH    DS2  GN_A05_BH : DS2 (1,25 - 2,00m)             1.25
2  GN_A05_BH    DS3  GN_A05_BH : DS3 (2,00 - 2,00m)             2.00
3  GN_A05_BH    DS4  GN_A05_BH : DS4 (3,16 - 4,00m)             3.16
4  GN_A05_BH    DS5  GN_A05_BH : DS5 (4,40 - 5,00m)             4.40
(3259, 4)


In [ ]:
# example: search for specific sample
# samples_df[samples_df['ID'].str.contains('GN_A05_BHA') & samples_df['ID'].str.contains('CR')]

In [255]:
# function to load txt with PSD data and extract info

def extract_psd_data(sample_txt):
    
    # dictionary to store individual row
    sample_dict = {}
    
    # open txt
    with open(sample_txt, 'r') as psd:

        # iterate over individual lines in file and check if it contains needed data
        # store data in sample_dict

        sample_dict['Test type'] = 'ds'

        for index, line in enumerate(psd):

            if 'Proben-Nr.:' in line:
                sample_dict['Location'] = line.split()[1]
                sample_dict['Sample'] = line.split()[3]

            if 'Tiefe [m]' in line:
                sample_dict['Depth'] = line.split()[-1].replace(',', '.')

            if 'Hydrometer' in line or 'Schlämmanalyse' in line:
                    sample_dict['Test type'] = 'wshy'

            if 'Clay:' in line or 'Ton:' in line:
                if line.split()[-1] == '-':
                    sample_dict['% clay [%]'] = 0.0
                else:
                    sample_dict['% clay [%]'] = line.split()[1]

            if 'Silt:' in line or 'Schluff:' in line:
                if line.split()[-1] == '-':
                    sample_dict['% silt [%]'] = 0.0
                else:
                    sample_dict['% silt [%]'] = line.split()[1]

            if 'Sand:' in line:
                if line.split()[-1] == '-':
                    sample_dict['% sand [%]'] = 0.0
                else:
                    sample_dict['% sand [%]'] = line.split()[1]

            if 'Stones:' in line or 'Steine:' in line:
                if line.split()[-1] == '-':
                    sample_dict['% gravel [%]'] = 0.0
                else:
                    sample_dict['% gravel [%]'] = line.split()[1]

            if 'Diameter for 10% passage' in line or 'Durchmesser bei 10% Durchgang' in line:
                if line.split()[-1] == 'mm':
                    sample_dict['D10 [mm]'] = line.split()[-2]
                else:
                    sample_dict['D10 [mm]'] = 'NaN'

            if 'Diameter for 30% passage' in line or 'Durchmesser bei 30% Durchgang' in line:
                if line.split()[-1] == 'mm':
                    sample_dict['D30 [mm]'] = line.split()[-2]
                else:
                    sample_dict['D30 [mm]'] = 'NaN'

            if 'Diameter for 60% passage' in line or 'Durchmesser bei 60% Durchgang' in line:
                if line.split()[-1] == 'mm':
                    sample_dict['D60 [mm]'] = line.split()[-2]
                else:
                    sample_dict['D60 [mm]'] = 'NaN'
                    
        return sample_dict


In [256]:
# create empty dataframe to store psd data

# psd_df = pd.DataFrame(columns = ['Location', 'Sample', 'Depth', 'Test type', '% clay [%]', '% silt [%]', '% sand [%]', '% gravel [%]', 'D10 [mm]', 'D30 [mm]', 'D60 [mm]'])
psd_df = pd.DataFrame()

directory_path = r'\\deberlfs04\Geoengineering\Baugrund\2022\Projekte_364\364_22_012_WPD_Germany_Gennaker SI\4_Data\1_Field and Laboratory Testing_Soil Mechanics\02 Laborergebisse\GN_A05_BHA\GN_A05_BHA_Excel_txt_LAB\GN_A05_BHA_KGV_txt'

# loop over GGU txt-files in directory

for file in os.scandir(directory_path):
    
    file_path = file.path
    
    if file_path.endswith(".txt"):
        
        sample_data = extract_psd_data(file_path)
        # convert dictionary to DataFrame
        sample_df = pd.DataFrame(sample_data, index=[0])

        # add sample to psd_df
        psd_df = pd.concat([psd_df, sample_df])


# print and check for errors
print(psd_df)

  Test type    Location  Sample  Depth D10 [mm] D30 [mm] D60 [mm] % clay [%]  \
0        ds  GN_A05_BHA  BAG-01   0.50  0.10650  0.14340  0.18193        0.0   
0        ds  GN_A05_BHA    CR08  10.64  0.13417  0.17470  0.26126        0.0   
0        ds  GN_A05_BHA  BAG-11  15.20  0.13046  0.16395  0.22973        0.0   
0        ds  GN_A05_BHA    CR14  19.76  0.12996  0.15443  0.19889        0.0   
0        ds  GN_A05_BHA  BAG-16  22.80  0.21390  0.32214  0.50096        0.0   
0      wshy  GN_A05_BHA    CR20  27.50      NaN  0.00485  0.13458       22.1   
0        ds  GN_A05_BHA    CR03   3.04  0.06339  0.08302  0.12824        0.0   
0        ds  GN_A05_BHA    CR04   4.56  0.06356  0.09347  0.15091        0.0   

  % silt [%] % sand [%]  % gravel [%]  
0        2.2       97.8           0.0  
0        0.9       99.1           0.0  
0        2.8       97.2           0.0  
0        2.2       97.8           0.0  
0        0.2       96.5           0.0  
0       29.6       28.5           0.0  

## Merging with samples dataframe to retrieve full sample name

In [258]:
merged_df = psd_df.merge(samples_df, on=['Location', 'Sample'], how='left')

# assert that no entries got lost during the merge
if merged_df['ID'].isnull().any():
    print('Error during merge. Check sample names and depths.\n')
else:
    print('All samples merged successfully.')
    
print(merged_df)

All samples merged successfully.
  Test type    Location   Sample  Depth D10 [mm] D30 [mm] D60 [mm] % clay [%]  \
0        ds  GN_A05_BHA   BAG-01   0.50  0.10650  0.14340  0.18193        0.0   
1        ds  GN_A05_BHA     CR08  10.64  0.13417  0.17470  0.26126        0.0   
2        ds  GN_A05_BHA   BAG-11  15.20  0.13046  0.16395  0.22973        0.0   
3        ds  GN_A05_BHA  CR13-14  19.76  0.12996  0.15443  0.19889        0.0   
4        ds  GN_A05_BHA   BAG-16  22.80  0.21390  0.32214  0.50096        0.0   
5      wshy  GN_A05_BHA     CR20  27.50      NaN  0.00485  0.13458       22.1   
6        ds  GN_A05_BHA     CR03   3.04  0.06339  0.08302  0.12824        0.0   
7        ds  GN_A05_BHA     CR04   4.56  0.06356  0.09347  0.15091        0.0   

  % silt [%] % sand [%]  % gravel [%]                                     ID  \
0        2.2       97.8           0.0     GN_A05_BHA : BAG-01 (0,50 - 1,20m)   
1        0.9       99.1           0.0     GN_A05_BHA : CR08 (10,64 - 12,16m) 

In [257]:
# space for potential corrections

# example: (wrong sample name)
# correct sample name:
# psd_df.iloc[3, 2] = 'CR13-14'

# !! merge again afterwards (cell above)

## Fill out missing columns and calculate values

In [259]:
# change data types to allow calculations

dtype_dict = {'% clay [%]' : float,
              '% silt [%]' : float,
              '% sand [%]' : float,
              '% gravel [%]' : float,
              '% gravel [%]' : float,
              'Depth' : float,
              'D10 [mm]' : float,
              'D30 [mm]' : float,
              'D60 [mm]' : float,
             }

merged_df = merged_df.astype(dtype_dict)

# check
merged_df.dtypes

Test type           object
Location            object
Sample              object
Depth              float64
D10 [mm]           float64
D30 [mm]           float64
D60 [mm]           float64
% clay [%]         float64
% silt [%]         float64
% sand [%]         float64
% gravel [%]       float64
ID                  object
Sample from [m]    float64
dtype: object

In [260]:
## fill out further columns ##

# constant values
merged_df['Filter'] = 'f'
merged_df['Validity'] = 'v'
merged_df['Standard'] = 'ENISO'
merged_df['Laboratory'] = '980348e2-8d78-4ab6-b5d7-4d0c469302a1'
merged_df['Curve fitting method'] = 's'


## calculate values for missing columns ##

# % fines = % clay + % silt
merged_df['% fines [%]'] = merged_df['% clay [%]'] + merged_df['% silt [%]']
# Cc = D60 / D10
merged_df['Cc'] = merged_df['D60 [mm]'] / merged_df['D10 [mm]']
# Cu = D30² / (D10 * D60)
merged_df['Cu'] = merged_df['D30 [mm]'] ** 2 / (merged_df['D10 [mm]'] * merged_df['D60 [mm]'])
# add offset column
merged_df['Offset [cm]'] = (merged_df['Depth'] - merged_df['Sample from [m]']) * 100

In [264]:
# assert that depths are correct
if (merged_df['Offset [cm]'] < 0).any():
    print('Wrong offsets.')
else:
    print('Offsets look good.')
    
print(merged_df['Offset [cm]'])

Offsets look good.
0      0.0
1      0.0
2      0.0
3    152.0
4      0.0
5      0.0
6      0.0
7      0.0
Name: Offset [cm], dtype: float64


In [263]:
# space for potential corrections (again)

# example: (negative offset)
# merged_df.iloc[2, -1] = 0
# !! merge again afterwards (cell above)

In [265]:
# put columns into right order

# list of columns in right order
column_list = ['ID', 'Location', 'Sample', 'Depth', 'Sample from [m]', 'Offset [cm]',
               'Laboratory', 'Filter', 'Validity', 'Standard', 'Test type', 
               '% fines [%]', '% clay [%]', '% silt [%]', '% sand [%]', '% gravel [%]',
               'D10 [mm]', 'D30 [mm]', 'D60 [mm]', 'Cc', 'Cu',
               'Curve fitting method', 
              ]

# reorder df based on column list
df_export = merged_df[column_list]

# check
df_export

,ID,Location,Sample,Depth,Sample from [m],Offset [cm],Laboratory,Filter,Validity,Standard,...,% clay [%],% silt [%],% sand [%],% gravel [%],D10 [mm],D30 [mm],D60 [mm],Cc,Cu,Curve fitting method
0,"GN_A05_BHA : BAG-01 (0,50 - 1,20m)",GN_A05_BHA,BAG-01,0.50,0.50,0.0,980348e2-8d78-4ab6-b5d7-4d0c469302a1,f,v,ENISO,...,0.0,2.2,97.8,0.0,0.10650,0.14340,0.18193,1.708263,1.061315,s
1,"GN_A05_BHA : CR08 (10,64 - 12,16m)",GN_A05_BHA,CR08,10.64,10.64,0.0,980348e2-8d78-4ab6-b5d7-4d0c469302a1,f,v,ENISO,...,0.0,0.9,99.1,0.0,0.13417,0.17470,0.26126,1.947231,0.870678,s
2,"GN_A05_BHA : BAG-11 (15,30 - 15,70m)",GN_A05_BHA,BAG-11,15.20,15.30,0.0,980348e2-8d78-4ab6-b5d7-4d0c469302a1,f,v,ENISO,...,0.0,2.8,97.2,0.0,0.13046,0.16395,0.22973,1.760923,0.896866,s
3,"GN_A05_BHA : CR13-14 (18,24 - 21,28m)",GN_A05_BHA,CR13-14,19.76,18.24,152.0,980348e2-8d78-4ab6-b5d7-4d0c469302a1,f,v,ENISO,...,0.0,2.2,97.8,0.0,0.12996,0.15443,0.19889,1.530394,0.922658,s
4,"GN_A05_BHA : BAG-16 (22,80 - 23,40m)",GN_A05_BHA,BAG-16,22.80,22.80,0.0,980348e2-8d78-4ab6-b5d7-4d0c469302a1,f,v,ENISO,...,0.0,0.2,96.5,0.0,0.21390,0.32214,0.50096,2.342029,0.968446,s
5,"GN_A05_BHA : CR20 (27,50 - 28,75m)",GN_A05_BHA,CR20,27.50,27.50,0.0,980348e2-8d78-4ab6-b5d7-4d0c469302a1,f,v,ENISO,...,22.1,29.6,28.5,0.0,NaN,0.00485,0.13458,NaN,NaN,s
6,"GN_A05_BHA : CR03 (3,04 - 3,69m)",GN_A05_BHA,CR03,3.04,3.04,0.0,980348e2-8d78-4ab6-b5d7-4d0c469302a1,f,v,ENISO,...,0.0,9.5,90.5,0.0,0.06339,0.08302,0.12824,2.023032,0.847854,s
7,"GN_A05_BHA : CR04 (4,56 - 5,46m)",GN_A05_BHA,CR04,4.56,4.56,0.0,980348e2-8d78-4ab6-b5d7-4d0c469302a1,f,v,ENISO,...,0.0,9.6,90.4,0.0,0.06356,0.09347,0.15091,2.374292,0.910841,s


## Export to Excel

In [267]:
excel_path = os.path.join(directory_path, 'psd_export.xlsx')

df_export.to_excel(excel_path, index=False)